In [31]:
from stats_can import StatsCan
sc = StatsCan()
# Annual stock of active vehicle registrations
fleet = sc.table_to_df("23-10-0308-01")     # ≈ 80 k rows


g:\RepEV\EV_Efficacity_2025\.venv\Lib\site-packages\stats_can\api_class.py:24: FutureWarning: This class will be deprecated in upcoming v3 release. Please see the docs for details
  warn(


g:\RepEV\EV_Efficacity_2025\.venv\Lib\site-packages\stats_can\sc.py:608: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for details.
  warn(
g:\RepEV\EV_Efficacity_2025\.venv\Lib\site-packages\stats_can\sc.py:326: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for details.
  warn(


In [8]:
# pip install stats-can pandas
from stats_can import StatsCan
import pandas as pd
def get_fleet()-> pd.DataFrame:
    """
    Get the annual stock of active vehicle registrations in Canada."""
    sc = StatsCan()
    tbl = sc.table_to_df("23-10-0308-01")             # ≈ 80 k rows, cached

    # --- keep latest reference year and the total stock of vehicles ------------
    latest_year = tbl["REF_DATE"].max()               # 2023, as of May 2025
    fleet = tbl[(tbl["REF_DATE"] == latest_year)]

    # --- pivot into one row per province ---------------------------------------
    prov_fleet = (fleet[["GEO", "VALUE", "Vehicle Type", "Fuel Type"]]
                  .rename(columns={"GEO": "Province", "VALUE": "Vehicles", "Vehicle Type": "Type", "Fuel Type": "Fuel"})
                  .set_index("Province")
                  .sort_index())
    return prov_fleet


In [ ]:
from stats_can import StatsCan
import pandas as pd

# 1. pull the fleet table
sc    = StatsCan()
tbl   = sc.table_to_df("23-10-0308-01")
year  = tbl["REF_DATE"].max()

# 2. keep only the light-duty fleet, all fuel types
base = (
    tbl[(tbl["REF_DATE"] == year) &
        (tbl["Vehicle Type"] == "Light-duty vehicle")]
    [["GEO", "Vehicle Type", "Fuel Type", "VALUE"]]
    .rename(columns={
        "GEO":   "Region",
        "VALUE": "Count",
        "Fuel Type": "Fuel",
    })
)

stock = base.pivot_table(
    index=["Region","Vehicle Type", "Fuel Type"],
    columns="Fuel",
    values="Count",
    fill_value=0
).reset_index()



g:\RepEV\EV_Efficacity_2025\.venv\Lib\site-packages\stats_can\api_class.py:24: FutureWarning: This class will be deprecated in upcoming v3 release. Please see the docs for details
  warn(
g:\RepEV\EV_Efficacity_2025\.venv\Lib\site-packages\stats_can\sc.py:608: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for details.
  warn(
g:\RepEV\EV_Efficacity_2025\.venv\Lib\site-packages\stats_can\sc.py:326: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for details.
  warn(
C:\Users\lpgag\AppData\Local\Temp\ipykernel_24156\3154028926.py:21: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  stock = base.pivot_table(


In [18]:
def simulate_electrification(df, frac_e=1.0, frac_h=0.0, keep_non=0.0):
    """
    df : the pivoted stock DataFrame.
    frac_e : fraction of non-EV replaced by EV
    frac_h : fraction of non-EV replaced by HEV
    keep_non: fraction of non-EV left unchanged
    (must satisfy frac_e + frac_h + keep_non == 1)
    """
    out = df.copy()
    non_ev = out[out["Fuel"] == "All fuel types"] - out[out["Fuel"] == "Battery-electric"] - out[out["Fuel"]=="Hybrid-electric"]
    
    # how many get converted
    to_ev = frac_e * non_ev
    to_hev = frac_h * non_ev
    keep   = keep_non * non_ev

    out[out["Fuel"] =="Battery-electric"] += to_ev
    out[out["Fuel"] == "Hybrid-electric"] += to_hev
    out[out["Fuel"] == "Gasoline"]        = keep   # or whatever your “other” fuel column is

    return out


In [19]:
stock = get_fleet()  # run this to get the fleet data

# 100% electric replacement:
sim100e = simulate_electrification(stock, frac_e=1.0, frac_h=0.0, keep_non=0.0)

# 50/50 electric / hybrid:
sim50_50 = simulate_electrification(stock, frac_e=0.5, frac_h=0.5, keep_non=0.0)

# 100% hybrid:
sim100h = simulate_electrification(stock, frac_e=0.0, frac_h=1.0, keep_non=0.0)


g:\RepEV\EV_Efficacity_2025\.venv\Lib\site-packages\stats_can\api_class.py:24: FutureWarning: This class will be deprecated in upcoming v3 release. Please see the docs for details
  warn(
g:\RepEV\EV_Efficacity_2025\.venv\Lib\site-packages\stats_can\sc.py:608: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for details.
  warn(
g:\RepEV\EV_Efficacity_2025\.venv\Lib\site-packages\stats_can\sc.py:326: FutureWarning: This function will be deprecated in the v3 release. Please see the docs for details.
  warn(


ValueError: cannot reindex on an axis with duplicate labels